<a href="https://colab.research.google.com/github/lilyrausser/DS/blob/main/quantity_item_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Business Question:

predict the quantity of menu items that a customer will buy using someone’s Payment Type_Payment ID, Order_day, Race, ZipCode, IncomeStatus, FamilySize, Price of menu item, and average_review.

In [ ]:
pip install xgboost

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

Preprocessing

In [ ]:
query1 = pd.read_csv('query1.csv')
query1['Race']

def map_race(race):
    if race == 'Caucasian':
        return 1
    elif race == 'Asian':
        return 2
    elif race == 'African American':
        return 3
    elif race == 'Hispanic':
        return 4
    else:
        return 0

query1['Race'] = query1['Race'].map(map_race)
query1

,Payment_Type_PaymentID,MONTH(o.Order_date_modified),Race,ZipCode,Income_Status,Family_Size,average_review,price,Quantity
0,3,2,1,12345,High Income,3,4.8,5.49,4
1,2,1,2,23456,Middle Income,2,4.5,6.49,4
2,1,2,3,34567,Low Income,4,4.2,3.99,2
3,2,2,3,34567,Low Income,4,4.2,2.79,4
4,2,1,3,34567,Low Income,4,4.5,4.99,3
...,...,...,...,...,...,...,...,...,...
95,3,1,2,45678,High Income,4,4.3,2.99,1
96,3,2,4,67890,Low Income,3,4.5,4.29,2
97,3,1,1,78901,Middle Income,2,4.5,4.29,2
98,2,2,2,89012,High Income,4,4.8,1.99,1


In [ ]:
query1['Income_Status']

def map_income(status):
    if status == "Low Income":
      return 1
    if status == "Middle Income":
      return 2
    if status == "High Income":
      return 3

query1['Income_Status'] = query1['Income_Status'].map(map_income)
query1

,Payment_Type_PaymentID,MONTH(o.Order_date_modified),Race,ZipCode,Income_Status,Family_Size,average_review,price,Quantity
0,3,2,1,12345,3,3,4.8,5.49,4
1,2,1,2,23456,2,2,4.5,6.49,4
2,1,2,3,34567,1,4,4.2,3.99,2
3,2,2,3,34567,1,4,4.2,2.79,4
4,2,1,3,34567,1,4,4.5,4.99,3
...,...,...,...,...,...,...,...,...,...
95,3,1,2,45678,3,4,4.3,2.99,1
96,3,2,4,67890,1,3,4.5,4.29,2
97,3,1,1,78901,2,2,4.5,4.29,2
98,2,2,2,89012,3,4,4.8,1.99,1


In [ ]:
query1 = query1.rename(columns={"Payment_Type_PaymentID": "PaymentID", "MONTH(o.Order_date_modified)": "Month"})
query1

,PaymentID,Month,Race,ZipCode,Income_Status,Family_Size,average_review,price,Quantity
0,3,2,1,12345,3,3,4.8,5.49,4
1,2,1,2,23456,2,2,4.5,6.49,4
2,1,2,3,34567,1,4,4.2,3.99,2
3,2,2,3,34567,1,4,4.2,2.79,4
4,2,1,3,34567,1,4,4.5,4.99,3
...,...,...,...,...,...,...,...,...,...
95,3,1,2,45678,3,4,4.3,2.99,1
96,3,2,4,67890,1,3,4.5,4.29,2
97,3,1,1,78901,2,2,4.5,4.29,2
98,2,2,2,89012,3,4,4.8,1.99,1


Calculating VIF (variance inflation factor) to determine how correlated the columns are to each other:

In [ ]:
def VIF(df, columns):
    values = sm.add_constant(df[columns]).values
    num_columns = len(columns)+1
    vif = [variance_inflation_factor(values, i) for i in range(num_columns)]
    return pd.Series(vif[1:], index=columns)

cols = ['PaymentID', 'Month', 'Race', 'ZipCode', 'Income_Status', 'Family_Size', 'average_review', 'price', 'Quantity']

VIF(query1, cols)

PaymentID         1.056198
Month             1.078208
Race              1.167554
ZipCode           1.035865
Income_Status     1.177091
Family_Size       1.081295
average_review    1.134969
price             1.119673
Quantity          1.051167
dtype: float64

Good results, VIF sub 5 so not correlated

Begin Model

In [ ]:
X = query1[cols].drop('Quantity', axis=1)
y = query1['Quantity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)

xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 2.2809753191194475


MSE is best to calculate because accuracy is not important to track with continuous variables

value of 2.28 is a high MSE; with our quantity values between a range of 1 and 4, 2.28 is on the higher end [the mean of the model being 2].

If we had a wider range of data, we may be able to infer more valuable information from this business question.